In [1]:
import os, sys
from graphnet.data.sqlite.sqlite_utilities import create_table
import pandas as pd
import sqlite3
import pyarrow.parquet as pq
import sqlalchemy
from tqdm import tqdm
from typing import Any, Dict, List, Optional
import numpy as np
import gc

graphnet: INFO     2023-03-02 20:56:58 - get_logger - Writing log to logs/graphnet_20230302-205658.log
graphnet: WARNING  2023-03-02 20:56:59 - warn_once - `icecube` not available. Some functionality may be missing.


In [2]:
input_data_folder = './data/train'
meta_data_path = './data/train_meta.parquet'
geometry_table = pd.read_csv('./data/sensor_geometry_fe.csv')
database_path = './data/database_1.db'
engine = sqlalchemy.create_engine("sqlite:///" + database_path)

In [11]:
def load_input(meta_batch: pd.DataFrame, input_data_folder: str) -> pd.DataFrame:
        batch_id = meta_batch['batch_id'].unique()
        assert len(batch_id) == 1, "contains multiple batch_ids. Did you set the batch_size correctly?"
        
        detector_readings = pd.read_parquet(path = f'{input_data_folder}/batch_{batch_id[0]}.parquet')
        sensor_positions = geometry_table.loc[detector_readings['sensor_id'], ['x', 'y', 'z', 'scattering', 'absorption', 'f0', 'f1', 'f2', 'f3']]
        sensor_positions.index = detector_readings.index

        for column in sensor_positions.columns:
            if column not in detector_readings.columns:
                detector_readings[column] = sensor_positions[column]

        detector_readings['auxiliary'] = detector_readings['auxiliary'].replace({True: 1, False: 0})
        return detector_readings.reset_index()


In [12]:
def add_to_table(database_path: str,
                      df: pd.DataFrame,
                      table_name:  str,
                      is_primary_key: bool,
                      engine: sqlalchemy.engine.base.Engine) -> None:
                      
    try:
        create_table(   columns=  df.columns,
                        database_path = database_path, 
                        table_name = table_name,
                        integer_primary_key= is_primary_key,
                        index_column = 'event_id')
    except sqlite3.OperationalError as e:
        if 'already exists' in str(e):
            pass
        else:
            raise e
   
    df.to_sql(table_name, con=engine, index=False, if_exists="append", chunksize = 200000)
    engine.dispose()
    return

In [13]:
def convert_to_sqlite(meta_data_path: str,
                      database_path: str,
                      input_data_folder: str,
                      batch_size: int = 200000,
                      batch_ids: list = list(range(1,2,1)),
                      engine: sqlalchemy.engine.base.Engine = None
                      ) -> None:
    
    meta_data_iter = pq.ParquetFile(meta_data_path).iter_batches(batch_size = batch_size)
    batch_id = 1
    converted_batches = []
    for meta_data_batch in tqdm(meta_data_iter):
        if batch_id in batch_ids:
            meta_data_batch  = meta_data_batch.to_pandas()
            add_to_table(database_path = database_path,
                        df = meta_data_batch,
                        table_name='meta_table',
                        is_primary_key= True,
                        engine = engine)
            pulses = load_input(meta_batch=meta_data_batch, input_data_folder= input_data_folder)
            del meta_data_batch 
            add_to_table(database_path = database_path,
                        df = pulses,
                        table_name='pulse_table',
                        is_primary_key= False,
                        engine = engine)
            del pulses 
            converted_batches.append(batch_id)
        batch_id +=1
        if len(batch_ids) == len(converted_batches):
            break
        gc.collect()
    del meta_data_iter 
    print(f'Conversion Complete! Database available at\n {database_path}')

In [14]:
database_path = f'./data/batch_1.db'
engine = sqlalchemy.create_engine("sqlite:///" + database_path)
convert_to_sqlite(meta_data_path,
                database_path=database_path,
                input_data_folder=input_data_folder,
                batch_size=200000,
                batch_ids=[1],
                engine=engine)

0it [03:44, ?it/s]

Conversion Complete! Database available at
 ./data/batch_1.db


In [24]:
conn = sqlite3.connect(database_path)
c = conn.cursor()

c.execute("SELECT * FROM meta_table LIMIT 5;")
data = c.fetchall()

In [25]:
pd.DataFrame(data, columns = [description[0] for description in c.description])

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117


In [21]:
conn = sqlite3.connect(database_path)
c = conn.cursor()

c.execute("SELECT * FROM pulse_table LIMIT 5;")
data = c.fetchall()

In [23]:
pd.DataFrame(data, columns=[description[0] for description in c.description])

,event_id,sensor_id,time,charge,auxiliary,x,y,z,scattering,absorption,f0,f1,f2,f3
0,24,3918,5928,1.325,1,303.41,335.64,206.58,0.555613,0.464598,1,0,0,0
1,24,4157,6115,1.175,1,-145.45,374.24,212.73,0.293056,0.216352,1,0,0,0
2,24,3520,6492,0.925,1,505.27,257.88,-174.60,0.109034,0.102424,0,0,0,1
3,24,5041,6665,0.225,1,-9.68,-79.50,181.00,0.212990,0.224349,1,0,0,0
4,24,2948,8054,1.575,1,576.37,170.92,357.88,1.578789,1.616654,1,0,0,0
